# Multi-Agents Real-Time Weather Example by Using SuperAgentX with AWS Bedrock LLM function Calling Feature.

### Open Source Github Repository URL: https://github.com/superagentxai/superagentx

## What is SuperAgentX?

SuperAgentX is an advanced agentic AI framework designed to accelerate the development of Artificial General Intelligence (AGI). It provides a powerful, modular, and flexible platform for building autonomous AI agents capable of executing complex tasks with minimal human intervention.

Using the SuperAgentX agentic AI framework, function calling with AWS Bedrock LLMs is simplified.

The below example creates weather as a handler (Tool) and invokes using SuperAgentX's agent.

#### The Below example explains how to invoke Bedrock Converse API using the SuperAgentX framework.

In this example, the SuperAgentX Agent module can automatically invoke a mock weather handler (Tool). This function can be fully operational by integrating any weather service API. Later in the example, we connect to the Open-Meteo API.

![SuperAgentX Multi Agent](assets/superagentX-Example-Weather.svg)


In [ ]:
!pip install superagentx

In [85]:
from superagentx.agent import Agent
from superagentx.agent import Engine
from superagentx.agentxpipe import AgentXPipe
from superagentx.llm import LLMClient
from superagentx.prompt import PromptTemplate
from superagentx.memory import Memory

In [86]:
%%html
<style>
table {float:left}
</style>

<style>
  table {margin-left: 0 !important;}
</style>

Although this example leverages Claude 3 Sonnet, Bedrock supports many other models. The full list of models and supported features can be found [here](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). The models are invoked via `bedrock-runtime`.


**Best Practice**: Always set your AWS credentials as environment variables.

```
export AWS_ACCESS_KEY=<<`YOUR_ACCESS_KEY`>>
export AWS_SECRET_KEY=<<`YOUR_ACCESS_SECRET_KEY`>>
export AWS_REGION=<<`AWS_REGION`>>
```

#### LLM Config in SuperAgentX

LLM Configuration Specify ==> llm_config

| Name | Description | Data Type | Required | Example |
|:-----|:------------|:----------|:---------|:--------|
|`model`| AWS Bedrock supported [models](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)|str| Yes| 'model': 'anthropic.claude-3-5-sonnet-20240620-v1:0' |
|`llm_type`| LLM type - `bedrock`|str| Yes| 'llm_type':'bedrock'


 

In [87]:
llm_config = {'model': 'anthropic.claude-3-5-sonnet-20241022-v2:0', 'llm_type':'bedrock'}

llm_client: LLMClient = LLMClient(llm_config=llm_config)

### Handler (Tool) definition 

We define `WeatherHandler` as a class where individual handlers (tools) are defined as functions. Note that there is nothing specific to the model used or Bedrock in this definition.

In [88]:
import requests

from superagentx.handler.base import BaseHandler
from superagentx.handler.decorators import tool

class WeatherHandler(BaseHandler):

    @tool
    async def get_weather(self, latitude: str, longitude: str) -> dict:
        """
        Get the weather data based on given latitude & longitude.

        Args:
            @param latitude: latitude of the location
            @param longitude: longitude of the location

            @return result (Str): Return Fake Weather for the given city & name in Fahrenheit
        """

        url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        response = requests.get(url)
        return response.json()

    @tool
    async def get_lat_long(self, place: str) -> dict:

        """
        Get the coordinates of a city based on a location.

        Args:
            @param city (str): The place name

            @return result (Str): Return the real latitude & longitude for the given place.

        """

        header_dict = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
            "referer": 'https://www.guichevirtual.com.br'
        }
        url = "http://nominatim.openstreetmap.org/search"

        params = {'q': place, 'format': 'json', 'limit': 1}
        response = requests.get(url, params=params, headers=header_dict).json()
        if response:
            lat = response[0]["lat"]
            lon = response[0]["lon"]
            return {"latitude": lat, "longitude": lon}
        else:
            return None

### Create Prompt Template Object

In [89]:
system_prompt = """You're provided with a tool that can get the coordinates for a specific city 
        'get_lat_long' and a tool that can get the weather for that city, but requires the coordinates 'get_weather'; 
        only use the tool if required. You can call the tool multiple times in the same response if required. Don't 
        make reference to the tools in your final answer."""

weather_prompt = PromptTemplate(system_message=system_prompt)

### Handler Engine

Create `Engine` object to invoke `WeatherHandler` along with prompt template and LLM client of `Bedrock` converse API.

In [90]:
weather_forecast_engine = Engine(
            handler=WeatherHandler(),
            llm=llm_client,
            prompt_template=weather_prompt
)


## Agent attributes

| Attribute                  | Parameter  | Description                                                                                                                                                                                                                                    |
| :------------------------- | :--------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Role**                   | `role`     | Defines the agent's function within the SuperAgentX. It determines the kind of activcity the agent is best suited for.                                                                                                                                    |
| **Goal**                   | `goal`     | The individual objective that the agent aims to achieve. It guides the agent's decision-making process.                                                                                                                                        |
| **LLM**              | `llm`| Represents the language model that will run the agent.|
| **Retry Max** *(optional)*       | `max_retry`      | sets the maximum number of iterations an agent can complete before it must provide its best possible answer. Default `5`                                                       |
| **Prompt Template**       | `prompt_template`      | Specifies the prompt format for the agent. |                                                       |
| **Engine**    | `engines`      | A list of engines (or lists of engines) that the engine can utilize in `parallel` or `sequnetial`. This allows for flexibility in processing and task execution based on different capabilities or configurations or or configurations.|

#### Agent - Get latitude & longitude from Weather Handler using `engine`.

In [96]:
lat_and_long_agent = Agent(
            role="Weather Man",
            goal="Get the latitude and longitude for the given place",
            llm=llm_client,
            max_retry=2,  # Default Max Retry is 5
            prompt_template=weather_prompt,
            engines=[weather_forecast_engine],
)

#### Agent - Get real-time weather for the given latitude & longitude from Weather Handler using `engine`.


In [92]:
weather_agent_agent = Agent(
            role="Weather Man",
            goal="Get the real-time weather based on the given latitude and longitude.",
            llm=llm_client,
            max_retry=2,  # Default Max Retry is 5
            prompt_template=weather_prompt,
            engines=[weather_forecast_engine],
)

### Pipe Attributes

| Attribute                  | Parameter  | Description                                                                                                                                                                                                                                    |
| :------------------------- | :--------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Name**                   | `name`     | An optional name for the agentxpipe, providing a more friendly reference for display or logging purposes.                                                                                                                                    |
| **Agents**                   | `agents`     | A list of Agent instances (or lists of Agent instances) that are part of this structure. These agents can perform tasks and contribute to achieving the defined goal.         |


#### Pipe - Executes agents `sequential` and provide results.


In [93]:
pipe = AgentXPipe(
            name='Weather Man',
            agents=[lat_and_long_agent, weather_agent_agent]
)

#### Pipe Flow - Triggers the pipe flow based on the user input `query_instruction`

In [94]:
result = await pipe.flow(query_instruction="What is the weather in Las Vegas?")

#### Print Results. The result object contains array of `GoalResult`.

In [95]:
print(f"Weather result => \n{result}")

Weather result => 
[GoalResult(name='Agent-4996bc6bbb6749e1a70b165e32e017f9', agent_id='4996bc6bbb6749e1a70b165e32e017f9', reason='The output context contains the correct latitude and longitude coordinates for Las Vegas: latitude of 36.1672559 and longitude of -115.148516. This directly fulfills the goal of getting coordinates for the given place.', result={'latitude': '36.1672559', 'longitude': '-115.148516'}, content=None, error=None, is_goal_satisfied=True), GoalResult(name='Agent-4996bc6bbb6749e1a70b165e32e017f9', agent_id='4996bc6bbb6749e1a70b165e32e017f9', reason='The output context contains the required weather information for Las Vegas with specific coordinates (36.1672559, -115.148516). The data shows current temperature, wind speed, wind direction, and weather conditions.', result='Currently in Las Vegas, the temperature is 13.9°C with a wind speed of 10.7 km/h from the northwest (310°). The sky is clear (weathercode: 0).', content=None, error=None, is_goal_satisfied=True)]
